Chapter 2.b.iv.  Instantiation
=======

**Instantiation** is a powerful derivation rule that can transform judgments with great flexibility by implicitly invoking *Lambda application* (beta reduction).  It is based upon the fact that variables can represent anything and, therefore, we may replace each free variable in an existing judgment with any expression, as long as we do so consistently.  That is, we may replace any free variable on either side of the turnstile of a judgment with any expression, as long as we replace all occurrences of the free variable across the judgment in a consistent manner.  Implicitly, if a judgment is valid, any such transformation must be allowed.

To be more precise about the machinery, an ad-hoc Lambda expression is generated for each component of the judgment (each assumption as well as the right side) with the respective component becoming the body of the corresponding Lambda expression.  The parameters of each of these ad-hoc lambda expressions will be the same and are dictacted by which variables are to be replaced.  Then beta reduction is applied using operands dictated by the desired replacements, the same operands for each ad-hoc lambda expression, with all of the reduction rules discussed in previous chapters.  The reduced expressions form a new judgment.  For example, some $\{A(x)\} \vdash P(x)$ where $x$ is a variable can be transformed to some $\{A(t)\} \vdash P(t)$ where $t$ is an arbitrary expression via applying the ad-hoc lambda expressions, $x \mapsto A(x)$ and $x \mapsto P(x)$, to the operand $t$.

*Assumptions* may be provided when performing an instantiation.  These may be needed to satisfy *requirements* needed to validate any reductions when performing the lambda applications.  Such requirements then become requirements of the derivation step, creating additional dependencies (beyond that of the original judgment) in the proof graph.  Assumptions that were needed to fulfill these requirements will be added as assumptions in the new judgment.

Finally, and importantly, instantiation may be used to eliminate any number of outer universal quantifiers on the right side of the turnstile provided that the conditions of the quantifiers are satisfied under these replacements.  These also become *requirements* of the derivation step to be included in the proof graph and may need *assumptions* to be proven.  For example, we can instantiate $A \vdash \forall_{x~|~Q_1(x), Q_2(x)} P(x)$ to get $A \cup B \vdash P(t)$ provided $A \cup B \vdash Q_1(t)$ and $A \cup B \vdash Q_2(t)$ are provable.

In [ ]:
import proveit
%begin instantiation

## Instantiation as lambda applications (beta reductions)

First, let's prove a "judgment" that we can work with for demonstrating instantiation.

In [ ]:
from proveit._common_ import A, P, Q, R
from proveit.logic import Implies
P_impl_R = Implies(P, R).prove([Implies(P, Q), Implies(Q, R)])

The following is just a simple relabeling via `instantiate`, changing 
the label $P$ to $A$ on both sides of the judgment via supplying `{P:A}` as the `repl_map` dictionary in the call to `instantiate`.

In [ ]:
A_impl_R = P_impl_R.instantiate({P:A})

Internally, this is done by effectively generating the lambda expressions $P \mapsto (P \Rightarrow Q)$, $P \mapsto (Q \Rightarrow R)$, and $P \mapsto (P \Rightarrow R)$, applying each of these functions to the single operand $A$, and then replacing those respective beta reductions to the respective components of the original judgment.  The proof reveals the **instantiation** step which is followed by the replacement mapping, $P:A$, and requires the original judgment.

In [ ]:
A_impl_R.proof()

The following example is done by effectively generating the lambda expressions $(P, R) \mapsto (P \Rightarrow Q)$, $(P, R) \mapsto (Q \Rightarrow R)$, and $(P, R) \mapsto (P \Rightarrow R)$ and applying each of these functions to the operands $(A, A)$.

In [ ]:
A_impl_R = P_impl_R.instantiate({P:A, R:A})

We can `instantiate` to any expressions, not just variables.  For example

In [ ]:
from proveit import Lambda
from proveit._common_ import f, x, y, fx, fy
from proveit.numbers import subtract
fx_impl_R_weird = P_impl_R.instantiate({P:fx, Q:Lambda(y, subtract(y, x))})

This may seem like a silly statement, and it is.  The assumptions are absurd with Lambda expressions for an implication consequent and an implication antecedant that should certainly not evaluate to true or false.  **Prove-It** does not care, however.  It is making no presumption that the implication operands are Boolean values.  It is merely a statement that if you were to assume these absurd things as being true, you can logically derive that $f(x)$ implies $R$.  In Prove-It's universe, this is a fact under these assumptions.  To prove consistency, we need not worry about whether we can derive bogus facts from bogus assumptions (or bogus implication antecedents), we only need to ensure that we cannot derive bogus facts from valid assumptions.

## Instantiating ranges of free variables

In addition to instantiating any free variables of a given judgment, we can also instantiate a range of its free variables.  To demonstrate, let us first create a judgment with a range of free variables.

In [ ]:
from proveit.core_expr_types._common_ import x_1_to_n, f__x_1_to_n
f__x_1_to_n__impl_R = P_impl_R.instantiate({P:f__x_1_to_n})

Now let's prepare a replacement for this range of variables.

In [ ]:
from proveit import ExprRange, ExprTuple
from proveit.logic import InSet
from proveit._common_ import a, n
from proveit.numbers import Natural, Add, one
x_repl_1 = ExprRange(a, Add(a, a), one, n)

And now we may call `instantiate` to make this replacement.

In [ ]:
f__1p1_to_npn__impl_R = f__x_1_to_n__impl_R.instantiate(
    {ExprTuple(x_1_to_n):ExprTuple(x_repl_1)}, assumptions=[InSet(n, Natural)])

This was done internally by effectively generating the lambda expressions $(x_1, ..., x_n) \mapsto (f(x_1, .., x_n) \Rightarrow Q)$, $(x_1, ..., x_n) \mapsto (Q \Rightarrow R)$, and $(x_1, ..., x_n) \mapsto (f(x_1, .., x_n) \Rightarrow R)$, applying each of these functions to the operands $(1+1), ..., (n+n)$ under the assumption that $n \in \mathbb{N}$.  $n \in \mathbb{N}$ was added to the assumptions list and the proof reveals the extra requirement necessary to make sure that the number of parameters and operands do indeed match (step 2 of the proof).

In [ ]:
f__1p1_to_npn__impl_R.proof()

As we saw in a previous chapter with respect to lambda applications, we are allowed to split apart this range, supplying multiple operand entries for a single parameter entry as long as their lengths match.

In [ ]:
from proveit._common_ import a, k, m, x
from proveit.numbers import zero, Mult
x_repl_2 = ExprRange(a, Mult(a, a), zero, k), x, ExprRange(a, a, one, m)

In [ ]:
from proveit.core_expr_types import Len
from proveit.logic import Equals
assumption = Equals(Len(x_repl_2), Len(ExprRange(a, a, one, n)))

In [ ]:
arbitrary_fn__impl__R = f__x_1_to_n__impl_R.instantiate({ExprTuple(x_1_to_n):x_repl_2}, 
                                                        assumptions=[assumption])

### Instantiating with alternative expansions

We also saw in a previous chapter that we may optionally provide an `equiv_alt_expansions` dictionary when calling `Lambda.apply` to provide versatility when a variable occurs in various forms with respect to ranging over its indices.  To access this feature in an **instantiation**, simply provide the alternative expansions in the `repl_map`.  For example, let's convert the following expression (with partial masking of some of the $A$ variable indices) to a judgment by simply assuming it to be true.

In [ ]:
from proveit import IndexedVar, var_range
from proveit._common_ import A, B, C, D, i, j, k, m
from proveit.core_expr_types._common_ import A_i_to_j
from proveit.logic import Not, And, Or, Forall
from proveit.numbers import one, Neg, subtract, NaturalPos
A_1_to_j, A_m = var_range(A, one, j), IndexedVar(A, m)
B_i_to_j = var_range(B, i, j)
partially_masked_expr = And(A_1_to_j, Forall(A_i_to_j, Or(A_i_to_j)))

In [ ]:
partially_masked_judgment = partially_masked_expr.prove([partially_masked_expr])

And now we will prepare `operands` and an alternate form for $A_1, \ldots, A_k$, making some assumptions and proving some equivalences as needed.

In [ ]:
operands = ExprTuple(ExprRange(k, Not(IndexedVar(B, k)), one, subtract(i, one)), B_i_to_j)

In [ ]:
assumptions = [InSet(i, NaturalPos), InSet(j, NaturalPos), InSet(Add(j, Neg(i), one), Natural)]
#InSet(subtract(j, subtract(i, one)), Natural)]

In [ ]:
alt_A_form = ExprTuple(var_range(A, one, subtract(i, one)), A_i_to_j)

In [ ]:
alt_A_form.merger(assumptions)

In [ ]:
from proveit import extract_var_tuple_indices
extract_var_tuple_indices(alt_A_form).merger(assumptions)

Finally, we will call the `instantiate` method, providing an expansion for $(A_1, \ldots, A_j)$ as well as $(A_1, \ldots A_{i-1}, A_i, \ldots, A_j)$ (we simply use `operands` for both).

In [ ]:
partially_masked_judgment.instantiate({ExprTuple(A_1_to_j):operands, alt_A_form:operands},
                                        assumptions=assumptions)

Notice that we relabeled the inner masked portion, replacing the $\forall_{A_i, \ldots, A_j}$ with $\forall_{B_i, \ldots, B_j}$.  Instantiation always does this inner relabeling when it is valid to do so (it doesn't have the `allow_relabeling` flag that the `Lambda.apply` method has, but it isn't really necessary as we will see).  If the replacement for a masked portion is not a proper relabeling, it will simply skip it.

In [ ]:
operands = ExprTuple(ExprRange(k, Not(IndexedVar(B, k)), one, subtract(i, one)), 
                     ExprRange(k, Not(IndexedVar(B, k)), i, j))

In [ ]:
partially_masked_judgment.instantiate({ExprTuple(A_1_to_j):operands, alt_A_form:operands},
                                        assumptions=assumptions)

This kind of masking should generally be avoided in any case, but if there is no other way around the need to relabel an inner expression, the inner expression can always be substituted with a form that is proven to be equal to it but with different labels.  More typically, we can perform a simple variable relabeling of an inner expression to be discussed in more detail in Chapter 2.a.vi.

In [ ]:
partially_masked_judgment.inner_expr().operands[1].relabel({A:C})

### Instantiating operators with lambda expressions

The other reduction rules discussed in previous chapters are also applicable during **instantiation** of course.  For example, if we can now instantiate $f$ with a Lambda expression to invoke a reduction of the $f$ operations via an extra internal beta reduction.

In [ ]:
from proveit import Lambda
f_repl = Lambda(x_1_to_n, Add(x_1_to_n))

In [ ]:
arbitrary_fn__impl__R.instantiate({f:f_repl}, assumptions=[assumption])

Once again, we have a silly statement, but it is no less truthful as far as **Prove-It** is concerned.  The right side of the judgment does derive from its silly assumptions.

Rather than using a Lambda expression for the replacement, we may also use the implicit form $f(x_1, .., x_n) : x_1 + ... + x_n$.  This form is preferable, in fact, though internally it is transformed to $f : (x_1, .., x_n) \mapsto x_1 + ... + x_n$ as we had before.

In [ ]:
wacky = arbitrary_fn__impl__R.instantiate({f__x_1_to_n:Add(x_1_to_n)}, assumptions=[assumption])

Note, however, that the $f(x_1, .., x_n) : x_1 + ... + x_n$ form is used when the proof graph is displayed (see the line below step 0):

In [ ]:
wacky.proof()

### Eliminating universal quantifiers while instantiating

The above examples illustrate what can be done by instantiating free variables on either side of an existing judgment.  A more common and useful application of instantiation, however, is to eliminate universal quantifiers on the right side of an existing judgment.  For example,

In [ ]:
from proveit import Function
from proveit._common_ import x, Px, Qx
from proveit.logic import Forall
simple_univ_quant = Forall(x, Px, condition=Qx)

To make a convenient example, we will just prove this by assumption.

In [ ]:
simple_univ_quant_assumed = simple_univ_quant.prove(assumptions=[simple_univ_quant])

In [ ]:
simple_univ_quant_inst = simple_univ_quant_assumed.instantiate(
    {x:Add(a, x)}, assumptions=[Function(Q, Add(a, x))])

Since $x$ was in our `repl_map` and it is universally quantified directly on the right side of the turnstile in `simple_univ_quant_assumed`, it is the indication to the `instantiate` method that we wish to eliminate that universal quantifier.  In the process, we apply or replacement map on the $Q(x)$ condition to obtain $Q(a+x)$ which is a requirement for this derivation step.  In this example, we satisify this requirement by simply assuming it.  In the proof graph below, this requirement is shown explicitly as step 2.

In [ ]:
simple_univ_quant_inst.proof()

We can eliminate multiple universal quantifiers simultaneously if the are nested directly on the right side of the quantifier.  It will eliminate the nested quantifiers as deep as the deepest of the variables in the `repl_map`.  For example, consider

In [ ]:
from proveit._common_ import a, b, c, d
nested_univ_quants = Forall(a, Forall(b, Forall(c, Forall(d, Function(P, (a, b, c, d))))))

In [ ]:
nested_univ_quants.instantiate({a:x, c:x}, assumptions=[nested_univ_quants])

We eliminated down to the quantifier for $c$ which was the deepest of the variable in the `repl_map`.

### Eliminating universal quantifiers with ranges of parameters and conditions

When instantiating ranges of parameters in univeral quantifiers being eliminated, the relevant range of indices  is indicated by the quantifier and is therefore not needed explicitly in the `repl_map`.  For example, instead of $(x_1, ..., x_n) : ((1+1), ..., (n+n))$, you can use $x : ((1+1), ..., (n+n))$ when $\forall_{..., x_1, ..., x_n, ...}$ is being eliminated.  This is called an implicit instantiation.  In this next example, let us invoke a theorem that is actually useful rather than something silly.

In [ ]:
from proveit.core_expr_types.tuples._theorems_ import general_len

In [ ]:
general_len

Let us instantiate this for one of our previous example tuples.

In [ ]:
x_repl_2

In [ ]:
general_len

In [ ]:
from proveit.numbers import num, Neg
from proveit._common_ import i, j
general_len_inst = \
    general_len.instantiate(
    {n:num(3), f:[Lambda(k, Mult(k, k)), Lambda(k, k), Lambda(k, k)],
     i:[zero, one, one], j:[k, one, m]},
    assumptions=[InSet(k, Natural), InSet(m, Natural)])

Looking at the proof for this (below), there are a number of requirements that needed to be satisfied for the instantiation.  In addition tot the `general_len_val` theorem itself, we needed to prove length matches for the $f_1, ..., f_3$, $i_1, ..., i_3$, and $j_1, ..., j_3$ replacements, and we needed to prove the $(j_1 - i_1 +1) \in \mathbb{N}, ..., (j_n - i_n +1) \in \mathbb{N}$ conditions.  Finally, because $f_2(i_1), ..., f_2(j_2)$ became $1, ..., 1$ which reduced to simply $1$, we require the fact that $(1, ..., 1) = (1)$ (step 8).

In [ ]:
general_len_inst.proof()

Finally, let us check the treatment when some of the instantiated conditions of an eliminated universal quantifiction are contained within an `ExprRange`.  Consider this direct instantiation (eliminating universal quantifiers with no non-trivial replacements) of the `general_len` theorem.

In [ ]:
general_len_simple_inst = \
    general_len.instantiate({j:j}, assumptions=general_len.all_conditions())

We see in step 2 of the proof that the requirement is a conjunction of the conditions, proven directly by assumption.

In [ ]:
general_len_simple_inst.proof()

For a slightly less trivial example, let's do a partial expansion of $f$, $i$, and $j$ within `general_len`.  This time, we will instantiate `general_len` indirectly via the `Len.computation` method.  In fact, it is considered best practice when possible to perform instantiation indirectly via convenient methods and automation routines developed in **Prove-It** theory systems.  This will be discussed at greater length in the "theory systems" chapters.

First some prepartion work:

In [ ]:
f_range_of_ranges = general_len.instance_expr.instance_expr.lhs.operand[0]

In [ ]:
index_assumptions = general_len.instance_expr.conditions

In [ ]:
Len([n, one]).typical_eq()

In [ ]:
extended__f_range_of_ranges = ExprTuple(f_range_of_ranges, x)

Now we will call the `Len.computation` method to instantiate the `general_len` theorem indirectly.  We pass the relevant `assumptions` into this theory and here we will indicate that we do not want to automatically simplify the resulting length computation so we can see the direct application of theorem in the proof.

In [ ]:
Len(extended__f_range_of_ranges).computation(
    assumptions=[*index_assumptions, InSet(n, Natural)], simplify=False).proof()

Notice the conjunction of conditions being satisfied in step 6.

Let us be explicit about the algorithm applied when we perform an **instantiation** that eliminates univeral quantifiers, which may involve ranges of parameters.  As we described in the first part of this chapter, an ad-hoc lambda map is created and applied to different parts of the judgment.  This is done to transform assumptions as before.  On the right side of the turnstile, as we eliminate universal quantification, we first create and apply an ad-hoc lambda map for each of its parameter entries (in case there are indices of a parameter range that must be instantiated), then we create and apply an ad-hoc lambda map to each condition (adding those outcomes to our list of *requirements*).  After eliminating all of the desired universal quantifications in that manner, we then create and apply an ad-hoc lambda map to the remaining inner expression.  For the most part, we use the same parameters and operands for each of these ad-hoc lambda maps as before, but the following are exceptions: the ad-hoc lambda map for a parameter entry of an eliminated universal quantifier excludes any entry involving the corresponding parameter variable, and an implicit instantiation is converted to an explicit form after encountering the corresponding explicit range of parameters.

In the instantiation of the `general_len` theorem above, the following ad-hoc lambda maps were created (illustrating both of the exceptions in the changes of the lambda parameters):

$\left(n, f, i, j\right) \mapsto n \in \mathbb{N}^+ \\
\left(n, i, j\right) \mapsto f_{1}, \ldots, f_{n} \\
\left(n, f_{1}, \ldots, f_{n + 1}, j\right) \mapsto i_{1}, \ldots, i_{n} \\
\left(n, f_{1}, \ldots, f_{n + 1}, i_{1}, \ldots, i_{n + 1}\right) \mapsto j_{1}, \ldots, j_{n} \\
\left(n, f_{1}, \ldots, f_{n + 1}, i_{1}, \ldots, i_{n + 1}, j_{1}, \ldots, j_{n + 1}\right) \mapsto \left(\left(j_{1} - i_{1} + 1\right) \in \mathbb{N}\right) \land \ldots \land \left(\left(j_{n} - i_{n} + 1\right) \in \mathbb{N}\right) \\
\left(n, f_{1}, \ldots, f_{n + 1}, i_{1}, \ldots, i_{n + 1}, j_{1}, \ldots, j_{n + 1}\right) \mapsto \begin{array}{c} |\left(f_{1}(i_{1}), \ldots, f_{1}(j_{1}), \ldots\ldots, f_{n}(i_{n}), \ldots, f_{n}(j_{n})\right)| \\  = \left(\left(j_{1} - i_{1} + 1\right) + \ldots + \left(j_{n} - i_{n} + 1\right)\right) \end{array}
$



In [ ]:
%end instantiation

# Next chapter: <a href="tutorial04_relabeling.ipynb">ToDo</a>

## <a href="tutorial00_introduction.ipynb#contents">Table of Contents</a>